Hyperparameteroptimierung für Regression

Benötigte Module importieren und Datei laden. Die ersten Zeilen werden ausgegeben.

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

path = "../../Data/iris.csv"
data = pd.read_csv(path, delimiter=',')
print(data.head())
print("Empty columns: ", data.columns[data.isnull().any()])

In [ ]:
# Ausgabe der Korrelationen
correlations = data[data.columns].corr(numeric_only=True)
print('All correlations')
print('-' * 30)
correlations_abs_sum = correlations[correlations.columns].abs().sum()
print(correlations_abs_sum)
print('Weakest correlations')
print('-' * 30)
print(correlations_abs_sum.nsmallest(5))

Daten vorbereiten.

In [ ]:
# data['sepal.width'] könnte man evtl weglassen, hat geringe Korrelation -> testen
data = data.drop(['sepal.width'], axis = 1)

# Diese Spalte soll vorhergesagt werden 
col = data['sepal.length']
data = data.drop(['sepal.length'], axis = 1)

# führe OHE für diese Daten durch
conv_ohe = ['species']
data = pd.get_dummies(data, columns = conv_ohe, dtype=float)

# Erzeuge Objekt
s_scaler = StandardScaler()
# Spalten für StandardScaler
cols_to_s_scale = ['petal.length', 'petal.width']
data[cols_to_s_scale] = s_scaler.fit_transform(data[cols_to_s_scale])

In [ ]:
print(data)

KNN aufbauen

In [ ]:
# Aus den zwei Tabellen vier Tabellen erzeugen
train_data, test_data, train_col, test_col = train_test_split(data,col, test_size=0.2, random_state=42)

# Aufbau KNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation=tf.nn.relu, input_dim=data.shape[1]),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
# Konfiguration des Lernprozesses
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

Trainieren

In [ ]:
# 70 Durchläufe
model.fit(train_data, train_col, epochs=70)

Testen

In [ ]:
test_loss, test_mae = model.evaluate(test_data, test_col)
print('Test mae:', test_mae)